In [1]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

from libs.common import *
from config.config import *

c:\Users\Admin\Data\ads_fraud_detection


c:/Users/Admin/Data/ads_fraud_detection


# 3. Tiền xử lí dữ liệu

In [2]:
save_dir=f"{exps_dir}/exp1/exp_gancopula"

save_dir = f"{save_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [3]:
test_size=0.33
seed=42
data=pd.read_excel(f'{exps_dir}/exp1/data_EDA.xlsx')
data

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
0,Y,Middle,OH,250/500,1000,1406.91,0,MALE,MD,craft-repair,sleeping,husband,53300,0,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,Morning,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2002-2008,Y,17,10,2014,25,1,2015
1,Y,Middle,IN,250/500,2000,1197.22,5000000,MALE,MD,machine-op-inspct,reading,other-relative,0,0,Vehicle Theft,?,Minor Damage,Police,VA,Riverwood,Morning,1,?,0,0,?,5070,780,780,3510,Mercedes,E400,2002-2008,Y,27,6,2006,21,1,2015
2,Y,Young,OH,100/300,2000,1413.14,5000000,FEMALE,PhD,sales,board-games,own-child,35100,0,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,NY,Columbus,Morning,3,NO,2,3,NO,34650,7700,3850,23100,Dodge,RAM,2002-2008,N,6,9,2000,22,2,2015
3,Y,Middle,IL,250/500,2000,1415.74,6000000,FEMALE,PhD,armed-forces,board-games,unmarried,48900,-62400,Single Vehicle Collision,Front Collision,Major Damage,Police,OH,Arlington,Morning,1,?,1,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2018++,Y,25,5,1990,10,1,2015
4,Y,Middle,IL,500/1000,1000,1583.91,6000000,MALE,Associate,sales,board-games,unmarried,66000,-46000,Vehicle Theft,?,Minor Damage,NaN,NY,Arlington,Night,1,NO,0,1,NO,6500,1300,650,4550,Accura,RSX,2018++,N,6,6,2014,17,2,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,N,Middle,OH,500/1000,1000,1310.80,0,FEMALE,Masters,craft-repair,paintball,unmarried,0,0,Single Vehicle Collision,Front Collision,Minor Damage,Fire,NC,Northbrook,Night,1,YES,0,1,?,87200,17440,8720,61040,Honda,Accord,2002-2008,N,16,7,1991,22,2,2015
996,Y,Middle,IL,100/300,1000,1436.79,0,FEMALE,PhD,prof-specialty,sleeping,wife,70900,0,Single Vehicle Collision,Rear Collision,Major Damage,Fire,SC,Northbend,Night,1,YES,2,3,?,108480,18080,18080,72320,Volkswagen,Passat,2018++,N,5,1,2014,24,1,2015
997,Y,Young,OH,250/500,500,1383.49,3000000,FEMALE,Masters,armed-forces,bungie-jumping,other-relative,35100,0,Multi-vehicle Collision,Side Collision,Minor Damage,Police,NC,Arlington,Morning,3,?,2,3,YES,67500,7500,7500,52500,Suburu,Impreza,1995-20001,N,17,2,2003,23,1,2015
998,Y,Old,IL,500/1000,2000,1356.92,5000000,MALE,Associate,handlers-cleaners,base-jumping,wife,0,0,Single Vehicle Collision,Rear Collision,Major Damage,Other,NY,Arlington,Morning,1,?,0,1,YES,46980,5220,5220,36540,Audi,A5,1995-20001,N,18,11,2011,26,2,2015


In [4]:
data_train, data_test = train_test_split(data, test_size = 0.33, random_state=seed)
print(data_train.shape,data_test.shape)
data_train.to_excel(f'{save_dir}/datatrain.xlsx', index=None)
data_test.to_excel(f'{save_dir}/datatest.xlsx', index=None)

(670, 40) (330, 40)


In [5]:
data_train

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
703,Y,Middle,IL,250/500,1000,817.28,0,MALE,High School,sales,cross-fit,unmarried,62200,0,Multi-vehicle Collision,Rear Collision,Total Loss,Ambulance,SC,Columbus,Evening,3,?,1,1,NO,60190,4630,9260,46300,Mercedes,ML350,2018++,Y,21,6,1994,3,1,2015
311,N,Young,IN,100/300,2000,765.64,0,MALE,MD,sales,base-jumping,not-in-family,0,-22200,Multi-vehicle Collision,Front Collision,Major Damage,Ambulance,SC,Northbrook,Morning,3,?,0,3,?,31350,2850,5700,22800,Honda,Accord,1995-20001,Y,11,6,2008,16,2,2015
722,Y,Middle,OH,100/300,2000,1422.21,0,FEMALE,Associate,handlers-cleaners,polo,unmarried,0,-46400,Single Vehicle Collision,Side Collision,Major Damage,Ambulance,WV,Riverwood,Night,1,YES,2,1,NO,85900,17180,17180,51540,Suburu,Forrestor,2002-2008,Y,29,7,1996,16,1,2015
629,Y,Middle,IL,100/300,2000,1133.27,0,FEMALE,High School,craft-repair,chess,wife,0,-56400,Multi-vehicle Collision,Side Collision,Total Loss,Other,NY,Northbrook,Night,2,?,0,2,YES,60500,11000,5500,44000,Ford,F150,1995-20001,Y,28,5,2014,1,3,2015
0,Y,Middle,OH,250/500,1000,1406.91,0,MALE,MD,craft-repair,sleeping,husband,53300,0,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,Morning,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2002-2008,Y,17,10,2014,25,1,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Y,Old,OH,500/1000,1000,1125.37,0,FEMALE,Associate,prof-specialty,basketball,husband,0,-56400,Multi-vehicle Collision,Rear Collision,Major Damage,Police,VA,Northbend,Morning,3,?,0,2,YES,79800,6650,19950,53200,Saab,95,1995-20001,Y,7,10,1990,13,1,2015
270,Y,Old,OH,250/500,2000,1589.54,0,MALE,College,handlers-cleaners,camping,husband,55400,0,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,VA,Arlington,Morning,3,?,2,0,YES,85300,17060,8530,59710,Toyota,Highlander,2002-2008,N,15,4,2013,27,1,2015
860,Y,Middle,IN,100/300,1000,1023.11,0,FEMALE,MD,adm-clerical,golf,own-child,0,-45300,Multi-vehicle Collision,Rear Collision,Minor Damage,Other,NY,Northbend,Evening,3,NO,1,2,YES,58850,10700,10700,37450,Accura,MDX,1995-20001,N,16,11,2004,6,2,2015
435,N,Young,OH,250/500,500,1075.41,0,MALE,MD,machine-op-inspct,reading,wife,55200,0,Single Vehicle Collision,Side Collision,Total Loss,Police,PA,Northbend,Morning,1,NO,1,0,NO,73400,7340,7340,58720,Dodge,Neon,1995-20001,N,29,8,1999,15,2,2015


In [6]:
# outliers_list=dict(np.load(f'{save_dir}/outliers.npz'))['outliers']
# def remove_outliers_iqr(data, column):
#     Q1 = data[column].quantile(0.25)
#     Q3 = data[column].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

# # Áp dụng hàm cho các cột chứa dữ liệu cần loại bỏ ngoại lệ
# for column in outliers_list:
#     data_train = remove_outliers_iqr(data_train, column)
# data_train

In [7]:
data.isnull().sum()

months_as_customer              4
age                             2
policy_state                    0
policy_csl                      0
policy_deductable               0
policy_annual_premium           0
umbrella_limit                  0
insured_sex                     0
insured_education_level         0
insured_occupation              0
insured_hobbies                 0
insured_relationship            0
capital-gains                   0
capital-loss                    0
incident_type                   0
collision_type                  0
incident_severity               0
authorities_contacted          91
incident_state                  0
incident_city                   0
incident_hour_of_the_day       52
number_of_vehicles_involved     0
property_damage                 0
bodily_injuries                 0
witnesses                       0
police_report_available         0
total_claim_amount              0
injury_claim                    0
property_claim                  0
vehicle_claim 

In [8]:
# data_train['collision_type']=data_train['collision_type'].replace('?',np.nan)
# data_train['collision_type'].fillna(method='ffill',inplace=True)
# data_train[['property_damage','police_report_available']]=data_train[['property_damage','police_report_available']].replace('?','Unknow')
# (data_train=='?').sum()

# 4.Chuẩn hóa dữ liệu

In [9]:
numeric_columns=dict(np.load(f'{exps_dir}/exp1/columns_dtype.npz',allow_pickle=True))['numeric_columns']
numeric_columns

array(['policy_annual_premium', 'capital-gains', 'capital-loss',
       'total_claim_amount', 'injury_claim', 'property_claim',
       'vehicle_claim', 'day_policy_bind_date', 'month_policy_bind_date',
       'year_policy_bind_date', 'day_incident_date',
       'month_incident_date', 'year_incident_date'], dtype='<U22')

In [10]:
category_columns=dict(np.load(f'{exps_dir}/exp1/columns_dtype.npz',allow_pickle=True))['category_columns']
category_columns

array(['months_as_customer', 'age', 'policy_state', 'policy_csl',
       'policy_deductable', 'umbrella_limit', 'insured_sex',
       'insured_education_level', 'insured_occupation', 'insured_hobbies',
       'insured_relationship', 'incident_type', 'collision_type',
       'incident_severity', 'authorities_contacted', 'incident_state',
       'incident_city', 'incident_hour_of_the_day',
       'number_of_vehicles_involved', 'property_damage',
       'bodily_injuries', 'witnesses', 'police_report_available',
       'auto_make', 'auto_model', 'auto_year', 'fraud_reported'],
      dtype='<U27')

In [11]:
label_encoders = {}
for column in category_columns:
    label_encoder = LabelEncoder()
    data_train[column] = label_encoder.fit_transform(data_train[column])
    label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

scaler = StandardScaler()
data_train[numeric_columns] = scaler.fit_transform(data_train[numeric_columns])
data_train

,months_as_customer,age,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,day_policy_bind_date,month_policy_bind_date,year_policy_bind_date,day_incident_date,month_incident_date,year_incident_date
703,1,0,0,1,1,-1.770734,0,1,2,11,6,4,1.270697,0.970481,0,2,2,0,4,1,0,2,0,1,1,1,0.249230,-0.612776,0.358110,0.416329,8,24,2,1,0.579574,-0.179439,-1.020460,-1.393943,-0.947412,0.0
311,0,2,1,0,2,-1.979562,0,1,4,11,0,1,-0.919382,0.175400,0,1,0,0,4,4,1,2,0,0,3,0,-0.845210,-0.976821,-0.374783,-0.834115,6,6,0,1,-0.547936,-0.179439,0.908584,0.115223,0.953085,0.0
722,1,0,2,0,2,0.675544,0,0,0,5,14,4,-0.919382,-0.691310,2,3,0,0,6,5,2,0,2,2,1,1,1.224890,1.953951,1.988592,0.695152,11,15,1,1,1.481582,0.108179,-0.744883,0.115223,-0.947412,0.0
629,1,0,0,0,2,-0.492901,0,0,2,2,5,5,-0.919382,-1.049454,0,3,2,2,1,4,2,1,0,0,2,2,0.260994,0.690017,-0.415957,0.293945,5,14,0,1,1.368831,-0.467057,1.735317,-1.626123,2.853582,0.0
0,1,0,2,1,1,0.613672,0,1,4,2,17,0,0.957325,0.970481,2,3,0,3,4,1,1,0,2,1,2,2,0.682604,-0.228278,1.132177,0.723886,10,1,1,1,0.128570,0.971032,1.735317,1.160031,-0.947412,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,1,1,2,2,1,-0.524848,0,0,0,9,1,0,-0.919382,-1.049454,0,2,0,3,5,3,1,2,0,0,2,2,0.993403,-0.199645,2.558849,0.783481,10,3,0,1,-0.998940,0.971032,-1.571616,-0.233046,-0.947412,0.0
270,1,1,2,1,2,1.352210,0,1,1,5,4,0,1.031267,0.970481,0,2,1,3,5,0,1,2,0,2,0,2,1.202121,1.929408,0.207826,1.129881,12,18,1,0,-0.096932,-0.754675,1.597528,1.392211,-0.947412,0.0
860,1,0,1,0,1,-0.938378,0,0,4,0,9,3,-0.919382,-0.651914,0,2,1,2,1,3,0,2,1,1,2,2,0.198379,0.628661,0.654562,-0.054583,0,23,0,0,0.015819,1.258650,0.357428,-1.045674,0.953085,0.0
435,0,2,2,1,0,-0.726882,0,1,4,6,15,5,1.024225,0.970481,2,3,2,3,3,3,1,0,1,1,0,1,0.750532,-0.058526,-0.037158,1.077203,4,27,0,0,1.481582,0.395797,-0.331516,-0.000866,0.953085,0.0


In [12]:
x_train=data_train.drop('fraud_reported',axis=1)
y_train=data_train['fraud_reported']
x_train.to_excel(f'{save_dir}/x_train.xlsx', index=False)
y_train.to_excel(f'{save_dir}/y_train.xlsx', index=False)

In [13]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict

{0: 0.6555772994129159, 1: 2.106918238993711}

In [14]:
# from imblearn.over_sampling import SMOTE
# x_train, y_train = SMOTE().fit_resample(x_train, y_train)
# x_train.to_excel(f'{save_dir}/x_train.xlsx', index=False)
# y_train.to_excel(f'{save_dir}/y_train.xlsx', index=False)

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_rosampled, y_rosampled = ros.fit_resample(data_train.drop(columns=['fraud_reported']), data_train['fraud_reported'])
X_rosampled = pd.DataFrame(X_rosampled)
y_rosampled = pd.DataFrame(y_rosampled)
oversampling_sample=pd.concat([X_rosampled, y_rosampled], axis=1)    

In [ ]:
from sdv.metadata import SingleTableMetadata
metadata_sync = SingleTableMetadata()
metadata_sync.detect_from_dataframe(data=oversampling_sample)

In [ ]:
from sdv.single_table import CopulaGANSynthesizer


gan_synthesizer = CopulaGANSynthesizer(metadata_sync,
                                            enforce_min_max_values=True,  
                                            enforce_rounding=True,
                                            default_distribution='gamma') 

gan_synthesizer.fit(oversampling_sample)
gan_synthesizer.save(
    filepath=f'{save_dir}/gan_copula_synthesizer.pkl'
)

In [ ]:
gan_synthetic_data = gan_synthesizer.sample(num_rows=1000)
gan_synthetic_data.head()

In [ ]:
label_encoders = {}
for column in category_columns:
    label_encoder = LabelEncoder()
    gan_synthetic_data[column] = label_encoder.fit_transform(gan_synthetic_data[column])
    label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

scaler = StandardScaler()
gan_synthetic_data[numeric_columns] = scaler.fit_transform(gan_synthetic_data[numeric_columns])


In [ ]:
from sdv.evaluation.single_table import run_diagnostic



diagnostic_gan = run_diagnostic(
    real_data=oversampling_sample,
    synthetic_data=gan_synthetic_data,
    metadata=metadata_sync
)




In [ ]:
from sdv.evaluation.single_table import evaluate_quality



diagnostic_gan = evaluate_quality(
    real_data=oversampling_sample,
    synthetic_data=gan_synthetic_data,
    metadata=metadata_sync
)




In [ ]:
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

## gan data
gan_data=gan_synthetic_data[gan_synthetic_data.fraud_reported==1]
gan_sample=pd.concat([data_train, gan_data] , axis=0, ignore_index=True)
X_gansampled, y_gansampled = rus.fit_resample(gan_sample.drop(columns=['fraud_reported']), gan_sample['fraud_reported'])
X_gansampled = pd.DataFrame(X_gansampled)
y_gansampled = pd.DataFrame(y_gansampled)
gan_sample=pd.concat([X_gansampled, y_gansampled], axis=1)    
gan_sample = shuffle(gan_sample, random_state=42)
print(gan_sample.fraud_reported.value_counts())

In [ ]:
gan_sample.to_excel(f'{save_dir}/data_train.xlsx', index=False)

In [ ]:
feat_gan = gan_sample.drop(columns=['fraud_reported'])
target_gan = gan_sample['fraud_reported']

In [ ]:
feat_gan.to_excel(f'{save_dir}/x_train.xlsx', index=False)
target_gan.to_excel(f'{save_dir}/y_train.xlsx', index=False)

In [16]:
object_cols_test=dict(np.load(f'{exps_dir}/exp1/columns_dtype.npz',allow_pickle=True))['category_columns']
numeric_cols_test=dict(np.load(f'{exps_dir}/exp1/columns_dtype.npz',allow_pickle=True))['numeric_columns']
# for column in outliers_list:
#     data_test = remove_outliers_iqr(data_test, column)
# data_test['collision_type'].replace('?',np.nan,inplace=True)
# data_test['collision_type'].fillna(method='ffill',inplace=True)
# data_test[['property_damage','police_report_available']].replace('?','Unknow',inplace=True)
# (data_test=='?').sum()
label_encoders = {}
for column in object_cols_test:
    label_encoder = LabelEncoder()
    data_test[column] = label_encoder.fit_transform(data_test[column])
    label_encoders[column] = label_encoder  # Lưu trữ label encoder nếu cần sau này

scaler = StandardScaler()
data_test[numeric_cols_test] = scaler.fit_transform(data_test[numeric_cols_test])

x_test=data_test.drop('fraud_reported',axis=1)
y_test=data_test['fraud_reported']
data_test.to_excel(f'{save_dir}/df_test_minmax.xlsx', index=False)
x_test.to_excel(f'{save_dir}/x_test.xlsx', index=False)
y_test.to_excel(f'{save_dir}/y_test.xlsx', index=False)